In [2]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            print(self.case.get("initial_weights"))
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [32]:
import tensorflow as tf

class TensorFlowModel: 
    def __init__(self, inputs, targets, learning_rate, batch_size, initial_weights, layers) -> None:
        self.inputs = inputs
        self.targets = targets
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.input_size = len(inputs[0])
        self.initial_weights = initial_weights
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)
        self.model = tf.keras.Sequential()
        
        for i, layer in enumerate(layers):
            if i == 0:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    input_shape=(self.input_size,),
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
            else:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
        
        loss_function = tf.keras.losses.CategoricalCrossentropy() if layers[-1]['activation_function'] == 'softmax' else tf.keras.losses.MeanSquaredError()
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    def fit_model(self, max_epochs, error_threshold):
        class ThresholdCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, _, logs = None):
                if(logs.get("loss") < error_threshold):
                    self.model.stop_training = True
        thressholdCallback = ThresholdCallback()
        self.model.fit(self.dataset, epochs=max_epochs, callbacks=[thressholdCallback])

    def show_prediction(self):
        print("============================= Result =============================\n")
        if(self.model.stop_training):
            print(f"Stopped by : error_threshold\n")
        else:
            print("Stopped by : max_iteration\n")
        for i,layer in enumerate(self.model.layers):
            weights, biases = layer.get_weights()
            print(f"Layer-{i}")
            print(f"Weights : {weights}\n")
            print(f"Biases : {biases}\n")
        print("==================================================================")


In [34]:
import os

def main():
    currDir = os.getcwd()
    testDir = currDir.replace("src","test")
    file = os.path.join(testDir, "softmax.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    print("ini inputs : ", inputs)
    targets = json_file.target
    learning_rate = json_file.learning_rate
    layers = json_file.layers
    batch_size = json_file.batch_size
    initial_weights = json_file.initial_weights
    max_iter = json_file.max_iteration
    error_threshold = json_file.error_threshold
    model = TensorFlowModel(inputs, targets, learning_rate, batch_size, initial_weights, layers)
    model.fit_model(max_iter, error_threshold)
    model.show_prediction()
main()

[[[0.1, 0.9, -0.1], [-0.2, 0.8, 0.2], [0.3, -0.7, 0.3], [0.4, 0.6, -0.4], [0.5, 0.5, 0.5], [-0.6, 0.4, 0.6], [-0.7, -0.3, 0.7], [0.8, 0.2, -0.8], [0.9, -0.1, 0.0]]]
ini inputs :  [[-2.4, -2.78, -0.6, 0.37, 2.46, -0.92, 2.76, 2.62], [-1.79, 1.65, -0.77, -1.03, 0.1, 2.12, -2.36, 1.25], [1.65, 2.34, 0.27, 2.34, 0.52, 1.37, 1.77, 0.62]]
Epoch 1/10


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 2.5703  
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 2.1917
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 1.8861
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 1.6386
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7083 - loss: 1.4317
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 1.2526
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7083 - loss: 1.0935
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7083 - loss: 0.9503
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 0.8212
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7083 - loss: 0.7057
============================= Result =============================

Stopped by : max_iteration

Layer-0
Weights : [[-0.33551654  0.67700493  0.45851156]
 [ 0.4831443  -0.852412

In [14]:
import tensorflow as tf

# Define the inputs and targets
inputs = tf.constant([
    [3.0, 1.0],
    [1.0, 2.0]
], dtype=tf.float32)

targets = tf.constant([
    [2.0, 0.3, -1.9],
    [1.3, -0.7, 0.1]
], dtype=tf.float32)

# Number of neurons, activation, and learning rate
number_of_neurons = 3
activation = "linear"
learning_rate = 0.1
batch_size = 2  # as you mentioned, batch_size of 2

# Initial weights and new uniform bias of 1
initial_weights = [
    [0.1, 0.3, 0.2],  # weights for inputs to neuron 1
    [0.4, 0.2, -0.7]  # weights for inputs to neuron 2
]
initial_bias = [1.0, 1.0, 1.0]  # uniform bias for all neurons

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(number_of_neurons, activation=activation, input_shape=(2,), use_bias=True)
])

# Set the initial weights and bias
model.layers[0].set_weights([tf.constant(initial_weights, dtype=tf.float32), tf.constant(initial_bias, dtype=tf.float32)])

# Compile the model with the specified optimizer and learning rate
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

# Since this is a simple forward pass, we'll use predict instead of fit to demonstrate batching
outputs = model.predict(inputs, batch_size=batch_size)

print("Outputs:")
print(outputs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Outputs:
[[ 1.7         2.1         0.90000004]
 [ 1.9000001   1.7        -0.19999993]]
